In [20]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from scipy.optimize import minimize

In [21]:
tickers = [f"Ticker_{i}" for i in range(3)]
num_values = 10**6
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2
0,-9.632300,4.293649,1.141841
1,-2.633829,4.444358,-4.435730
2,6.473480,5.761077,4.397759


In [22]:
def create_functional_1(random_values):
    def wrapper(weights):
        weighted_annual_ratio_sum_df = random_values.mul(weights).sum(axis=1)

        weighted_expectation = weighted_annual_ratio_sum_df.mean()
        weighted_variance = weighted_annual_ratio_sum_df.var(ddof=0)
        return weighted_expectation / np.sqrt(weighted_variance)

    return wrapper


def create_functional_2(random_values):
    cov_matrix = random_values.cov(ddof=0).values
    np.fill_diagonal(cov_matrix, 0)
    
    def wrapper(weights):
        numerator = weights @ random_values.mean().values
        denominator_var = (weights ** 2) @ random_values.var(ddof=0).values
        denominator_cov = (weights @ cov_matrix) @ weights
        return numerator / np.sqrt(denominator_var + denominator_cov)

    return wrapper

In [23]:
def constraint(x):
    return sum(x) - 1

con = {"type": "eq", "fun": constraint}
bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

In [24]:
def generate_random_weights(vector_size):
    weights = np.random.rand(vector_size)
    weights /= np.sum(weights)
    return weights


def random_search_for_initial_weights(functional_to_minimize, n_tries, vector_size):
    best_func_value = np.inf
    for _ in tqdm(range(n_tries)):
        weights = generate_random_weights(vector_size)
        res = minimize(functional_to_minimize, weights, constraints=con, bounds=bounds)
        if res.fun < best_func_value:
            best_func_value = res.fun

    return best_func_value

### 3 columns

In [25]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 3/3 [00:09<00:00,  3.05s/it]

functional_1: -0.0015352546737217592


In [26]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 3/3 [00:05<00:00,  1.74s/it]

functional_2: -0.0015319700277264766


### 8 columns

In [27]:
tickers = [f"Ticker_{i}" for i in range(8)]
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2,Ticker_3,Ticker_4,Ticker_5,Ticker_6,Ticker_7
0,-7.464206,-9.024906,1.544249,5.482106,7.712808,3.595739,4.988692,-1.706010
1,-7.033378,-9.351468,1.790498,-5.500424,2.719637,4.911934,-7.674953,8.809840
2,-1.340068,-0.836213,7.058128,1.481024,-0.306522,7.448163,-4.051039,-3.930265


In [28]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 3/3 [00:48<00:00, 16.04s/it]

functional_1: -0.0020886317175286204


In [29]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 3/3 [00:44<00:00, 14.86s/it]

functional_2: -0.002087380714975811
